In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 5.3 MB/s eta 0:00:00


In [ ]:
!pip install catboost


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.8 MB/s eta 0:00:00


In [ ]:
!pip install scikeras

In [ ]:
# Import libraries
import optuna
import pandas as pd
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasClassifier

# Load datasets
main_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/10_TPC (Tripeptide Composition)/positive_main_tpc.csv")
main_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/10_TPC (Tripeptide Composition)/negative_main_tpc (1).csv")
validation_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/10_TPC (Tripeptide Composition)/positive_validation_tpc.csv")
validation_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/10_TPC (Tripeptide Composition)/negative_validation_tpc.csv")

# Prepare features and labels for training
X_train = pd.concat([main_p, main_n])
y_train = pd.concat([pd.Series(1, index=main_p.index), pd.Series(0, index=main_n.index)])


In [ ]:
# Define a cross-validation accuracy function
def cross_val_accuracy(model, X_train, y_train):
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    accuracies = cross_val_score(model, X_train, y_train, cv=cv, scoring='accuracy')
    return accuracies.mean()


In [ ]:
#Support Vector Machine (SVM)
def objective_svm(trial):
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly'])
    C = trial.suggest_loguniform('C', 1e-5, 1e5)
    gamma = trial.suggest_loguniform('gamma', 1e-5, 1e-1)

    model = SVC(kernel=kernel, C=C, gamma=gamma)
    return cross_val_accuracy(model, X_train, y_train)

study_svm = optuna.create_study(direction='maximize')
study_svm.optimize(objective_svm, n_trials=50)
print("SVM Best Parameters:", study_svm.best_params)
print("SVM Best Accuracy:", study_svm.best_value)


[I 2024-12-05 07:19:21,957] A new study created in memory with name: no-name-a2e3bd63-2fc2-494a-8021-e3f5f1cb1dd8
<ipython-input-10-cd4fae0f87b1>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-5, 1e5)
<ipython-input-10-cd4fae0f87b1>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  gamma = trial.suggest_loguniform('gamma', 1e-5, 1e-1)
[I 2024-12-05 07:20:13,117] Trial 0 finished with value: 0.5043177445611958 and parameters: {'kernel': 'poly', 'C': 0.13995568599300037, 'gamma': 0.0023905651778763108}. Best is trial 0 with value: 0.5043177445611958.
<ipython-input-10-cd4fae0f87b1>:4: FutureWarning: suggest_loguniform has been depreca

SVM Best Parameters: {'kernel': 'linear', 'C': 48.263304221599945, 'gamma': 0.00010210037129534728}
SVM Best Accuracy: 0.9329843125647477


In [ ]:
#Random Forest
def objective_rf(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    max_depth = trial.suggest_int('max_depth', 3, 30)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf
    )
    return cross_val_accuracy(model, X_train, y_train)

study_rf = optuna.create_study(direction='maximize')
study_rf.optimize(objective_rf, n_trials=50)
print("Random Forest Best Parameters:", study_rf.best_params)
print("Random Forest Best Accuracy:", study_rf.best_value)


[I 2024-12-05 07:48:50,901] A new study created in memory with name: no-name-d53f0a65-3d75-402e-b3a7-29527830383a
[I 2024-12-05 07:48:57,522] Trial 0 finished with value: 0.8032632825218293 and parameters: {'n_estimators': 105, 'max_depth': 5, 'min_samples_split': 17, 'min_samples_leaf': 3}. Best is trial 0 with value: 0.8032632825218293.
[I 2024-12-05 07:49:01,932] Trial 1 finished with value: 0.790380346307533 and parameters: {'n_estimators': 145, 'max_depth': 25, 'min_samples_split': 11, 'min_samples_leaf': 8}. Best is trial 0 with value: 0.8032632825218293.
[I 2024-12-05 07:49:05,517] Trial 2 finished with value: 0.782651324552316 and parameters: {'n_estimators': 237, 'max_depth': 24, 'min_samples_split': 13, 'min_samples_leaf': 9}. Best is trial 0 with value: 0.8032632825218293.
[I 2024-12-05 07:49:12,422] Trial 3 finished with value: 0.8419305904987422 and parameters: {'n_estimators': 201, 'max_depth': 14, 'min_samples_split': 6, 'min_samples_leaf': 3}. Best is trial 3 with value

Random Forest Best Parameters: {'n_estimators': 207, 'max_depth': 30, 'min_samples_split': 18, 'min_samples_leaf': 2}
Random Forest Best Accuracy: 0.8737013467515169


In [ ]:
#Decision Tree
def objective_dt(trial):
    max_depth = trial.suggest_int('max_depth', 3, 30)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

    model = DecisionTreeClassifier(
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf
    )
    return cross_val_accuracy(model, X_train, y_train)

study_dt = optuna.create_study(direction='maximize')
study_dt.optimize(objective_dt, n_trials=50)
print("Decision Tree Best Parameters:", study_dt.best_params)
print("Decision Tree Best Accuracy:", study_dt.best_value)


[I 2024-12-05 07:54:22,477] A new study created in memory with name: no-name-b3118cf1-dbf1-454a-829d-3b3fdcdcbd00
[I 2024-12-05 07:54:25,129] Trial 0 finished with value: 0.8101561343791623 and parameters: {'max_depth': 23, 'min_samples_split': 12, 'min_samples_leaf': 10}. Best is trial 0 with value: 0.8101561343791623.
[I 2024-12-05 07:54:26,686] Trial 1 finished with value: 0.8015687435252332 and parameters: {'max_depth': 14, 'min_samples_split': 4, 'min_samples_leaf': 10}. Best is trial 0 with value: 0.8101561343791623.
[I 2024-12-05 07:54:28,432] Trial 2 finished with value: 0.8316079621133639 and parameters: {'max_depth': 19, 'min_samples_split': 16, 'min_samples_leaf': 1}. Best is trial 2 with value: 0.8316079621133639.
[I 2024-12-05 07:54:29,904] Trial 3 finished with value: 0.8075884268166345 and parameters: {'max_depth': 18, 'min_samples_split': 11, 'min_samples_leaf': 10}. Best is trial 2 with value: 0.8316079621133639.
[I 2024-12-05 07:54:31,429] Trial 4 finished with value:

Decision Tree Best Parameters: {'max_depth': 25, 'min_samples_split': 5, 'min_samples_leaf': 1}
Decision Tree Best Accuracy: 0.8556459967441172


In [ ]:
#k-Nearest Neighbors (k-NN)
def objective_knn(trial):
    n_neighbors = trial.suggest_int('n_neighbors', 3, 15)
    weights = trial.suggest_categorical('weights', ['uniform', 'distance'])
    algorithm = trial.suggest_categorical('algorithm', ['auto', 'ball_tree', 'kd_tree', 'brute'])

    model = KNeighborsClassifier(
        n_neighbors=n_neighbors,
        weights=weights,
        algorithm=algorithm
    )
    return cross_val_accuracy(model, X_train, y_train)

study_knn = optuna.create_study(direction='maximize')
study_knn.optimize(objective_knn, n_trials=50)
print("k-NN Best Parameters:", study_knn.best_params)
print("k-NN Best Accuracy:", study_knn.best_value)


[I 2024-12-05 07:56:15,372] A new study created in memory with name: no-name-1c7dca11-0d88-4ddd-b3c3-75f1a3f008ad
[I 2024-12-05 07:56:17,159] Trial 0 finished with value: 0.868554832026047 and parameters: {'n_neighbors': 11, 'weights': 'uniform', 'algorithm': 'auto'}. Best is trial 0 with value: 0.868554832026047.
[I 2024-12-05 07:56:20,372] Trial 1 finished with value: 0.835922006807755 and parameters: {'n_neighbors': 14, 'weights': 'uniform', 'algorithm': 'auto'}. Best is trial 0 with value: 0.868554832026047.
[I 2024-12-05 07:56:42,184] Trial 2 finished with value: 0.8711484386562084 and parameters: {'n_neighbors': 6, 'weights': 'uniform', 'algorithm': 'kd_tree'}. Best is trial 2 with value: 0.8711484386562084.
[I 2024-12-05 07:57:04,137] Trial 3 finished with value: 0.8342126683439396 and parameters: {'n_neighbors': 15, 'weights': 'uniform', 'algorithm': 'kd_tree'}. Best is trial 2 with value: 0.8711484386562084.
[I 2024-12-05 07:57:24,042] Trial 4 finished with value: 0.8805645996

k-NN Best Parameters: {'n_neighbors': 8, 'weights': 'distance', 'algorithm': 'kd_tree'}
k-NN Best Accuracy: 0.8908872280597899


In [ ]:
#Logistic Regression

def objective_lr(trial):
    C = trial.suggest_loguniform('C', 1e-5, 1e5)
    penalty = trial.suggest_categorical('penalty', ['l2'])
    solver = trial.suggest_categorical('solver', ['lbfgs', 'saga'])

    model = LogisticRegression(C=C, penalty=penalty, solver=solver)
    return cross_val_accuracy(model, X_train, y_train)

study_lr = optuna.create_study(direction='maximize')
study_lr.optimize(objective_lr, n_trials=50)
print("Logistic Regression Best Parameters:", study_lr.best_params)
print("Logistic Regression Best Accuracy:", study_lr.best_value)


[I 2024-12-05 08:11:21,705] A new study created in memory with name: no-name-a3bbd980-745c-409a-9078-e700f97084c6
<ipython-input-15-5e81b06dce9f>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-5, 1e5)
[I 2024-12-05 08:11:35,141] Trial 0 finished with value: 0.5473397957673524 and parameters: {'C': 0.0002127077791640454, 'penalty': 'l2', 'solver': 'saga'}. Best is trial 0 with value: 0.5473397957673524.
<ipython-input-15-5e81b06dce9f>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-5, 1e5)
[I 2024-12-05 08:11:39,750] Trial 1 finished with value: 0.9235311528784965 and parameters: {'C': 1603.1425

Logistic Regression Best Parameters: {'C': 8592.945154613994, 'penalty': 'l2', 'solver': 'lbfgs'}
Logistic Regression Best Accuracy: 0.927834097972473


In [ ]:
#Naive Bayes
def objective_nb(trial):
    var_smoothing = trial.suggest_loguniform('var_smoothing', 1e-9, 1e-1)

    model = GaussianNB(var_smoothing=var_smoothing)
    return cross_val_accuracy(model, X_train, y_train)

study_nb = optuna.create_study(direction='maximize')
study_nb.optimize(objective_nb, n_trials=50)
print("Naive Bayes Best Parameters:", study_nb.best_params)
print("Naive Bayes Best Accuracy:", study_nb.best_value)


[I 2024-12-05 08:23:39,454] A new study created in memory with name: no-name-e3fee3ed-fd79-4688-a490-d51ccc5b2861
<ipython-input-16-5c7e648b4d2b>:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  var_smoothing = trial.suggest_loguniform('var_smoothing', 1e-9, 1e-1)
[I 2024-12-05 08:23:41,292] Trial 0 finished with value: 0.902094124611514 and parameters: {'var_smoothing': 7.38865547373151e-06}. Best is trial 0 with value: 0.902094124611514.
<ipython-input-16-5c7e648b4d2b>:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  var_smoothing = trial.suggest_loguniform('var_smoothing', 1e-9, 1e-1)
[I 2024-12-05 08:23:43,021] Trial 1 finished with value: 0.9038145626757437 and par

Naive Bayes Best Parameters: {'var_smoothing': 9.015400221269638e-05}
Naive Bayes Best Accuracy: 0.9106741157318335


In [ ]:
#Gradient Boosting
def objective_gb(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    max_depth = trial.suggest_int('max_depth', 3, 20)
    subsample = trial.suggest_uniform('subsample', 0.5, 1.0)

    model = GradientBoostingClassifier(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        subsample=subsample
    )
    return cross_val_accuracy(model, X_train, y_train)

study_gb = optuna.create_study(direction='maximize')
study_gb.optimize(objective_gb, n_trials=50)
print("Gradient Boosting Best Parameters:", study_gb.best_params)
print("Gradient Boosting Best Accuracy:", study_gb.best_value)


[I 2024-12-05 08:25:17,059] A new study created in memory with name: no-name-5b38b904-3408-4548-b5db-37170876bceb
<ipython-input-17-0f44ec352f28>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
<ipython-input-17-0f44ec352f28>:6: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  subsample = trial.suggest_uniform('subsample', 0.5, 1.0)
[I 2024-12-05 08:25:58,341] Trial 0 finished with value: 0.820449163830102 and parameters: {'n_estimators': 77, 'learning_rate': 8.98610057254595e-05, 'max_depth': 11, 'subsample': 0.6131056344002606}. Best is trial 0 with value: 0.820449163830102.
<ipython-input-17-0f44ec352f28>:4: FutureWar

Gradient Boosting Best Parameters: {'n_estimators': 188, 'learning_rate': 0.043520571319760545, 'max_depth': 20, 'subsample': 0.8809690724874394}
Gradient Boosting Best Accuracy: 0.9037775640076957


In [ ]:
#XGBoost

def objective_xgb(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    subsample = trial.suggest_uniform('subsample', 0.5, 1.0)

    model = XGBClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        subsample=subsample
    )
    return cross_val_accuracy(model, X_train, y_train)

study_xgb = optuna.create_study(direction='maximize')
study_xgb.optimize(objective_xgb, n_trials=50)
print("XGBoost Best Parameters:", study_xgb.best_params)
print("XGBoost Best Accuracy:", study_xgb.best_value)


[I 2024-12-05 10:29:09,323] A new study created in memory with name: no-name-417479d8-2ccd-47c4-8075-cbba710da956
<ipython-input-18-ad894fc3f237>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
<ipython-input-18-ad894fc3f237>:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  subsample = trial.suggest_uniform('subsample', 0.5, 1.0)
[I 2024-12-05 10:30:15,116] Trial 0 finished with value: 0.8496596122539589 and parameters: {'n_estimators': 97, 'max_depth': 9, 'learning_rate': 0.033181426225409545, 'subsample': 0.8861669174067233}. Best is trial 0 with value: 0.8496596122539589.
<ipython-input-18-ad894fc3f237>:6: FutureWa

XGBoost Best Parameters: {'n_estimators': 151, 'max_depth': 16, 'learning_rate': 0.0962352815493093, 'subsample': 0.7586713539812318}
XGBoost Best Accuracy: 0.8943355039218588


In [ ]:
# LightGBM
def objective_lgb(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    num_leaves = trial.suggest_int('num_leaves', 2, 50)

    model = LGBMClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        num_leaves=num_leaves
    )
    return cross_val_accuracy(model, X_train, y_train)

study_lgb = optuna.create_study(direction='maximize')
study_lgb.optimize(objective_lgb, n_trials=50)
print("LightGBM Best Parameters:", study_lgb.best_params)
print("LightGBM Best Accuracy:", study_lgb.best_value)


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
#CatBoost
def objective_cb(trial):
    depth = trial.suggest_int('depth', 3, 12)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    l2_leaf_reg = trial.suggest_loguniform('l2_leaf_reg', 1e-2, 1e2)

    model = CatBoostClassifier(
        depth=depth,
        learning_rate=learning_rate,
        l2_leaf_reg=l2_leaf_reg,
        verbose=0
    )
    return cross_val_accuracy(model, X_train, y_train)

study_cb = optuna.create_study(direction='maximize')
study_cb.optimize(objective_cb, n_trials=50)
print("CatBoost Best Parameters:", study_cb.best_params)
print("CatBoost Best Accuracy:", study_cb.best_value)


[I 2024-12-05 11:35:04,369] A new study created in memory with name: no-name-8cf52cdb-a370-40de-917c-e104b8c6520b
<ipython-input-20-c756e5183119>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
<ipython-input-20-c756e5183119>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  l2_leaf_reg = trial.suggest_loguniform('l2_leaf_reg', 1e-2, 1e2)
[I 2024-12-05 11:36:11,427] Trial 0 finished with value: 0.8994894183809382 and parameters: {'depth': 5, 'learning_rate': 0.0376012495358659, 'l2_leaf_reg': 7.217964921073795}. Best is trial 0 with value: 0.8994894183809382.
<ipython-input-20-c756e5183119>:4: FutureW

In [ ]:
#AdaBoost
def objective_ab(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)

    model = AdaBoostClassifier(
        n_estimators=n_estimators,
        learning_rate=learning_rate
    )
    return cross_val_accuracy(model, X_train, y_train)

study_ab = optuna.create_study(direction='maximize')
study_ab.optimize(objective_ab, n_trials=50)
print("AdaBoost Best Parameters:", study_ab.best_params)
print("AdaBoost Best Accuracy:", study_ab.best_value)


In [ ]:
#Neural Network
def objective_nn(trial):
    # Define the hyperparameters
    num_layers = trial.suggest_int('num_layers', 1, 3)
    hidden_units = trial.suggest_int('hidden_units', 32, 512, step=32)
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
    epochs = trial.suggest_int('epochs', 10, 50)

    # Define the neural network
    model = Sequential()
    model.add(Dense(hidden_units, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dropout(dropout_rate))

    for _ in range(num_layers - 1):
        model.add(Dense(hidden_units, activation='relu'))
        model.add(Dropout(dropout_rate))

    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # Train the model
    history = model.fit(X_train, y_train,
                        validation_data=(X_val, y_val),
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=0)

    # Evaluate the model
    val_accuracy = max(history.history['val_accuracy'])
    return val_accuracy

# Run Optuna study
study_nn = optuna.create_study(direction='maximize')
study_nn.optimize(objective_nn, n_trials=50)
print("Neural Network Best Parameters:", study_nn.best_params)
print("Neural Network Best Accuracy:", study_nn.best_value)


In [ ]:
#Multi-Layer Perceptron (MLP)

def objective_mlp(trial):
    # Define the hyperparameters
    num_layers = trial.suggest_int('num_layers', 1, 3)
    hidden_units = trial.suggest_int('hidden_units', 32, 512, step=32)
    activation = trial.suggest_categorical('activation', ['relu', 'tanh', 'elu'])
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
    epochs = trial.suggest_int('epochs', 10, 50)

    # Define the MLP model
    model = MLPClassifier(hidden_layer_sizes=(hidden_units,) * num_layers,
                          activation=activation,
                          learning_rate_init=learning_rate,
                          max_iter=epochs,
                          batch_size=batch_size,
                          solver='adam',
                          random_state=42)

    # Evaluate using cross-validation
    scores = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy')
    return scores.mean()

# Run Optuna study
study_mlp = optuna.create_study(direction='maximize')
study_mlp.optimize(objective_mlp, n_trials=50)
print("MLP Best Parameters:", study_mlp.best_params)
print("MLP Best Accuracy:", study_mlp.best_value)


probabilites dataset

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Combine data and create labels
X_train = pd.concat([main_p, main_n], axis=0)
y_train = np.array([1] * len(main_p) + [0] * len(main_n))

X_validation = pd.concat([validation_p, validation_n], axis=0)
y_validation = np.array([1] * len(validation_p) + [0] * len(validation_n))

# Define models
models = {
    "RandomForest": RandomForestClassifier(random_state=42),
    "GradientBoosting": GradientBoostingClassifier(random_state=42),
    "LogisticRegression": LogisticRegression(max_iter=1000, random_state=42),
    "DecisionTree": DecisionTreeClassifier(random_state=42),
    "NaiveBayes": GaussianNB(),
    "k-NN": KNeighborsClassifier(),
    "SVM": SVC(probability=True, random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
    "LightGBM": LGBMClassifier(random_state=42),
    "CatBoost": CatBoostClassifier(verbose=0, random_state=42),
    "AdaBoost": AdaBoostClassifier(random_state=42),
    "MLP": MLPClassifier(max_iter=500, random_state=42)
}

# Combined probabilities dataset
combined_probabilities = pd.DataFrame(index=X_validation.index)

# Train models and get probabilities
for model_name, model in models.items():
    print(f"Training {model_name}...")
    model.fit(X_train, y_train)
    probabilities = model.predict_proba(X_validation)[:, 1]
    combined_probabilities[model_name] = probabilities

# Neural Network
def build_nn(input_dim):
    model = Sequential([
        Dense(128, activation='relu', input_dim=input_dim),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Train Neural Network
input_dim = X_train.shape[1]
nn_model = build_nn(input_dim)
nn_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_validation, y_validation), verbose=1)

# Predict probabilities for Neural Network
nn_probabilities = nn_model.predict(X_validation).flatten()
combined_probabilities['NeuralNetwork'] = nn_probabilities

# Add ground truth
combined_probabilities['True_Label'] = y_validation

# Save the combined probabilities dataset
output_path = "/content/drive/MyDrive/Cell penetrating peptide/Probabilities/OPTUNA With TPC.csv"
combined_probabilities.to_csv(output_path, index=False)
print(f"Combined probabilities dataset with Neural Network saved to {output_path}")
